<a href="https://colab.research.google.com/github/MasoudMoeini/Image-blur-detection/blob/main/image_blur_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
%load_ext tensorboard
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import matplotlib . pyplot as plt
from tensorflow.keras import layers, losses
# Base CNN
x = tf.placeholder(tf.float32, shape=[None, 224, 224, 3]) # input image
y = tf.placeholder(tf.float32, shape=[None, 224, 224, 3]) # Blur area detected

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [6]:
#!unzip -qq BlurDatasetResultShi.zip
!unzip -qq ccv_data.zip

In [63]:
rm -rf ./logs/

In [64]:
# Conv1
# Input Tensor Shape: [batch_size, 224, 224, 3]
# Output Tensor Shape: [batch_size, 224, 228, 32]
conv1 = tf.layers.conv2d(x, filters=32, kernel_size=[3,3], padding="same", activation=tf.nn.relu)

# conv2
# Input Tensor Shape: [batch_size, 224, 224, 32]
# Output Tensor Shape: [batch_size, 224, 228, 32]
conv2 = tf.layers.conv2d(conv1, filters=32, kernel_size=[3,3], padding="same", activation=tf.nn.relu)

# pool1
# Input Tensor Shape: [batch_size, 224, 224, 32]
# Output Tensor Shape: [batch_size, 112, 112, 32]
pool1 = tf.layers.max_pooling2d(conv2, pool_size=[2,2], strides=2, padding="same")

#conv3
# Input Tensor Shape: [batch_size, 112, 112, 32]
# Output Tensor Shape: [batch_size, 112, 112, 64]
conv3 = tf.layers.conv2d(conv2, filters=64, kernel_size=[3,3], padding="same", activation=tf.nn.relu)

#conv4
# Input Tensor Shape: [batch_size, 112, 112, 64]
# Output Tensor Shape: [batch_size, 112, 112, 64]
conv4 = tf.layers.conv2d(conv3, filters=64, kernel_size=[3,3], padding="same", activation=tf.nn.relu)

#pool2
# Input Tensor Shape: [batch_size, 112, 112, 64]
# Output Tensor Shape: [batch_size, 56, 56, 64]
pool2 = tf.layers.max_pooling2d(conv4, pool_size=[2,2], strides=2, padding="same")

#conv5
# Input Tensor Shape: [batch_size, 56, 56, 64]
# Output Tensor Shape: [batch_size, 56, 56, 128]
conv5 = tf.layers.conv2d(pool2, filters=128, kernel_size=[3,3], padding="same", activation=tf.nn.relu)

#conv6
# Input Tensor Shape: [batch_size, 56, 56, 128]
# Output Tensor Shape: [batch_size, 56, 56, 128]
conv6 = tf.layers.conv2d(conv5, filters=128, kernel_size=[3,3], padding="same", activation=tf.nn.relu)


#pool3
# Input Tensor Shape: [batch_size, 56, 56, 128]
# Output Tensor Shape: [batch_size, 28, 28, 128]
pool3 = tf.layers.max_pooling2d(conv6, pool_size=[2,2], strides=2, padding="same")

#conv7
# Input Tensor Shape: [batch_size, 28, 28, 128]
# Output Tensor Shape: [batch_size, 28, 28, 256]
conv7 = tf.layers.conv2d(pool3, filters=256, kernel_size=[3,3], padding="same", activation=tf.nn.relu)
#conv8
# Input Tensor Shape: [batch_size, 28, 28, 256]
# Output Tensor Shape: [batch_size, 28, 28, 256]
conv8 = tf.layers.conv2d(conv7, filters=128, kernel_size=[3,3], padding="same", activation=tf.nn.relu)

#pool4
# Input Tensor Shape: [batch_size, 28, 28, 256]
# Output Tensor Shape: [batch_size, 14, 14, 256]

pool4 = tf.layers.max_pooling2d(conv8, pool_size=[2,2], strides=2, padding="same")
#conv9
# Input Tensor Shape: [batch_size, 14, 14, 256]
# Output Tensor Shape: [batch_size, 14, 14, 512]
conv9 = tf.layers.conv2d(pool4, filters=512, kernel_size=[3,3], padding="same", activation=tf.nn.relu)

#pool5
# Input Tensor Shape: [batch_size, 14, 14, 512]
# Output Tensor Shape: [batch_size, 7, 7, 512]
pool5 = tf.layers.max_pooling2d(conv9, pool_size=[2,2], strides=2, padding="same")

#dim = int(np.prod(pool5.get_shape()[1:])) #7*7*512
#fcl = tf.reshape(pool5, shape=[-1, dim], name ='fc1')#[batch_size,7*7*512]
# decoder

# Input Tensor Shape: [batch_size, 7, 7, 512]
# Output Tensor Shape: [batch_size, 14, 14, 512]
net=tf.layers.conv2d_transpose(pool5,512,[3, 3],strides = 2,padding='SAME')

# Input Tensor Shape: [batch_size, 7, 7, 512]
# Output Tensor Shape: [batch_size, 28, 28, 256]
net=tf.layers.conv2d_transpose(net,256,[3, 3],strides = 2,padding='SAME')

# Input Tensor Shape: [batch_size, 28, 28, 128]
# Output Tensor Shape: [batch_size, 224, 224, 128]
net=tf.layers.conv2d_transpose(net,128,[3, 3],strides = 4,padding='SAME')

# Input Tensor Shape: [batch_size, 224, 224, 128]
# Output Tensor Shape: [batch_size, 224, 224, 64]
net=tf.layers.conv2d_transpose(net,64,[3, 3],strides = 1,padding='SAME')

# Input Tensor Shape: [batch_size, 224, 224, 64]
# Output Tensor Shape: [batch_size, 224, 224, 32]
net=tf.layers.conv2d_transpose(net,32,[3, 3],strides = 1, padding='SAME', activation = tf.nn.tanh)

# Input Tensor Shape: [batch_size, 224, 224, 32]
# Output Tensor Shape: [batch_size, 224, 224, 3]
net=tf.layers.conv2d_transpose(net,3,[3, 3],strides = 1, padding='SAME', activation = tf.nn.tanh)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/convolutional.py:575: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/pooling.py:600: UserWarni

In [65]:
## Optimize
n_epochs = 100
learning_rate = 0.01
loss = tf.reduce_mean(tf.square(net - y))
optimizer = tf.train.AdamOptimizer(learning_rate)
train  = optimizer.minimize(loss)

In [66]:
from generator_images import load_images_from_folder
train_input_path= 'ccv_data/train/images/'
train_target_path= 'ccv_data/train/blur/'

train_input_images = load_images_from_folder(train_input_path)
train_target_images = load_images_from_folder(train_target_path)

 800 from ccv_data/train/images/ successfully uploaded
 800 from ccv_data/train/blur/ successfully uploaded


In [50]:
BATCH_SIZE = 10
import cv2
img = np.empty((BATCH_SIZE, 224, 224, 3), np.float32)
l = list(enumerate(train_input_images[0:BATCH_SIZE]))
len(l)


10

In [16]:
## Define the Xavier initialization
#xav_init =  tf.contrib.layers.xavier_initializer()
## Define the L2 regularizer
#l2_regularizer = tf.contrib.layers.l2_regularizer(l2_reg)

In [67]:
BATCH_SIZE = 10
### Number of batches :  length dataset / batch size
n_batches = len(train_input_images) // BATCH_SIZE
## Set params
n_epochs = 100
learning_rate = 0.01

In [68]:
init=tf.global_variables_initializer()
averageloss=[]
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        print('epoch ',epoch)
        DLOSS=[]
        for i in range(n_batches):
            #m=i*batch_size
            #batch_images=train_input_images[i*BATCH_SIZE : (i+1)*BATCH_SIZE]
            #batch_target=train_target_images [i*BATCH_SIZE : (i+1)*BATCH_SIZE]
            batch_images = train_input_images[i*BATCH_SIZE : (i+1)*BATCH_SIZE].reshape(BATCH_SIZE,244,244,3)
            batch_target = train_target_images [i*BATCH_SIZE : (i+1)*BATCH_SIZE].reshape(BATCH_SIZE,244,244,3)
            batch_loss = sess.run([loss, train], feed_dict={x: batch_images, y: batch_target})
            DLOSS.append(batch_loss)
            print('epoch {} batch number {}    batch loss: {}'.format(epoch,i,batch_loss))
        MeanDloss=np.mean(DLOSS)
        averageloss.append(np.mean(DLOSS))
        print(' Average batches loss: {} '.format(MeanDloss))
       

epoch  0


AttributeError: ignored